In [1]:
library(data.table)
library(enrichR)
library(ggplot2)
library(dplyr)
library(stringr)

interactions <- read.table("CT/ranked_interactions_across_ligand_cell_type.txt", header = TRUE)
interactions <- interactions %>% filter(fdr_ligand < 0.01)

malignant_receptors <- interactions %>% filter(cell_type_receptor == "Malignant")
malignant_ligands <- interactions %>% filter(cell_type_ligand == "Malignant")


Welcome to enrichR
Checking connection ... 

Enrichr ... 
Connection is Live!

FlyEnrichr ... 
Connection is Live!

WormEnrichr ... 
Connection is Live!

YeastEnrichr ... 
Connection is Live!

FishEnrichr ... 
Connection is Live!

OxEnrichr ... 
Connection is Live!


Attaching package: ‘dplyr’


The following objects are masked from ‘package:data.table’:

    between, first, last


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




In [2]:
pdf(paste0("all_gsea_CT_interactions.pdf"), width = 10, height = 10)


#'Malignant''T_cells''B_cells''Fibroblasts''Other''Endothelial''Myeloid'
IoI <- "T_cells"
matched1 <- malignant_receptors %>% filter(cell_type_ligand == IoI)
matched2 <- malignant_ligands %>% filter(cell_type_receptor == IoI)
matched <- rbind(matched1, matched2)
matched_brainmets <- matched %>% filter(log2FC_ligand > 0 )
matched_brainmets <- matched_brainmets %>% arrange(desc(log2FC_ligand))

matched_primary <- matched %>% filter(log2FC_ligand < 0 )
matched_primary <- matched_primary %>% arrange(log2FC_ligand)

# Initialize an empty list


genes <- list()

# Iterate through each row
for (i in 1:nrow(matched_primary)) {
  # Append ligand and receptor values to the list
  if (!(matched_primary$ligand[i][[1]] %in% genes)) {
    genes[[length(genes) + 1]] <- matched_primary$ligand[i][[1]]
    }
  if (!(matched_primary$receptor[i][[1]] %in% genes)) {

      genes[[length(genes) + 1]] <- matched_primary$receptor[i][[1]]
  }
}

# The list now contains all ligand and receptor values
genes <- genes[1:min(50, length(genes))]

# Convert to an array
# Note: Arrays are typically used for numerical data, so ensure this is appropriate for your data
genes <- array(genes, dim = c(length(genes)))

# If a vector is more suitable, just use the extracted list as a vector
genes <- unlist(genes)
genes# Load Enrichr
listEnrichrSites()
setEnrichrSite("Enrichr") # Human genes
websiteLive <- TRUE

dbs <- c("MSigDB_Hallmark_2020","GO_Biological_Process_2021")
if (websiteLive) {
  enriched <- enrichr(genes, dbs)
}
mut_enr_ch<-mutate(enriched[[1]], qscore = -log(Adjusted.P.value, base=10))
mut_enr_go<-mutate(enriched[[2]], qscore = -log(Adjusted.P.value, base=10))
dim(mut_enr_ch)
dim(mut_enr_go)
mut_enr_167 <- rbind.data.frame(mut_enr_ch,mut_enr_go)
mut_enr <- rbind.data.frame(mut_enr_ch,mut_enr_go)

dim(mut_enr)
write.csv(mut_enr,file="mut_enr_metagenes.csv")
cohort = "Interactions_Malignant_and_T_cells_Primary"
enrichdbs = "Cancer_Hallmarks_and_Gene_Ontology"
topn = "50"
h_mut_enr <- mut_enr[1:25,]#[1:500,]
h_mut_enr
ggp<- h_mut_enr %>%
  ggplot(aes(qscore, reorder(Term, qscore), fill = P.value)) +
  scale_fill_gradient(low = "red", high = "blue") +
  geom_bar(stat = "identity") +
  theme(legend.position = "bottom",
        axis.title.y = element_blank()
  ) +
  geom_text(
    aes(label = paste("P.val=", round(P.value,3))),
    color = "black",
    size = 4,
    hjust = 1, nudge_x = 2
  ) + theme_bw()
myggp<-ggp + scale_y_discrete(labels = function(x) str_wrap(x, width = 50))+ggtitle(paste(cohort,"\n",enrichdbs,": top = ",topn,"_genes")) +xlab("qscore") + ylab("Description")
print(myggp)
dev.off()`

ERROR: Error in parse(text = x, srcfile = src): <text>:79:10: unexpected INCOMPLETE_STRING
78: print(myggp)
79: dev.off()`
             ^


In [3]:
pdf(paste0("all_gsea_CT_interactions.pdf"), width = 10, height = 10)
listEnrichrSites()
setEnrichrSite("Enrichr") # Human genes
websiteLive <- TRUE

dbs <- c("MSigDB_Hallmark_2020","GO_Biological_Process_2021")

cell_types <- c("Malignant", "T_cells", "B_cells", "Fibroblasts", "Other", "Endothelial", "Myeloid")

# Loop through each cell type
for (IoI in cell_types) {
  # Filter for matched_primary and matched_brainmets
  matched1 <- malignant_receptors %>% filter(cell_type_ligand == IoI)
  matched2 <- malignant_ligands %>% filter(cell_type_receptor == IoI)
  matched <- rbind(matched1, matched2)

  matched_brainmets <- matched %>% filter(log2FC_ligand > 0) %>% arrange(desc(log2FC_ligand))
  matched_primary <- matched %>% filter(log2FC_ligand < 0) %>% arrange(log2FC_ligand)
  temp_i = "BrainMets"
  # Perform analysis and plotting for matched_brainmets and matched_primary
  for (matched_subset in list(matched_brainmets, matched_primary)) {
        # Initialize an empty list
        genes <- list()
        # Iterate through each row
        for (i in 1:nrow(matched_subset)) {
          # Append ligand and receptor values to the list
          if (!(matched_subset$ligand[i][[1]] %in% genes)) {
            genes[[length(genes) + 1]] <- matched_subset$ligand[i][[1]]
            }
          if (!(matched_subset$receptor[i][[1]] %in% genes)) {
              genes[[length(genes) + 1]] <- matched_subset$receptor[i][[1]]
          }
        }
        # The list now contains all ligand and receptor values
        genes <- genes[1:min(50, length(genes))]
        genes <- array(genes, dim = c(length(genes)))
        genes <- unlist(genes)
        
        if (websiteLive) {
          enriched <- enrichr(genes, dbs)
        }
        mut_enr_ch<-mutate(enriched[[1]], qscore = -log(Adjusted.P.value, base=10))
        mut_enr_go<-mutate(enriched[[2]], qscore = -log(Adjusted.P.value, base=10))
        mut_enr <- rbind.data.frame(mut_enr_ch,mut_enr_go) 
        dim(mut_enr)
        cohort = paste0("Interactions_Malignant_", IoI, "_", temp_i)
        enrichdbs = "Cancer_Hallmarks_and_Gene_Ontology"
        topn = "50"
        h_mut_enr <- mut_enr[1:25,]#[1:500,]
        h_mut_enr
        ggp<- h_mut_enr %>%
          ggplot(aes(qscore, reorder(Term, qscore), fill = P.value)) +
          scale_fill_gradient(low = "red", high = "blue") +
          geom_bar(stat = "identity") +
          theme(legend.position = "bottom",
                axis.title.y = element_blank()
          ) +
          geom_text(
            aes(label = paste("P.val=", round(P.value,3))),
            color = "black",
            size = 4,
            hjust = 1, nudge_x = 2
          ) + theme_bw()
        myggp<-ggp + scale_y_discrete(labels = function(x) str_wrap(x, width = 50))+ggtitle(paste(cohort,"\n",enrichdbs,": top = ",topn,"_genes")) +xlab("qscore") + ylab("Description")
        print(myggp)
        Sys.sleep(3)

        temp_i <- "Primary"
    }
}
dev.off()

Enrichr ... 
Connection is Live!

FlyEnrichr ... 
Connection is Live!

WormEnrichr ... 
Connection is Live!

YeastEnrichr ... 
Connection is Live!

FishEnrichr ... 
Connection is Live!

OxEnrichr ... 
Connection is Live!

Connection changed to https://maayanlab.cloud/Enrichr/

Connection is Live!



Uploading data to Enrichr... Done.
  Querying MSigDB_Hallmark_2020... Done.
  Querying GO_Biological_Process_2021... Done.
Parsing results... Done.
Uploading data to Enrichr... Done.
  Querying MSigDB_Hallmark_2020... Done.
  Querying GO_Biological_Process_2021... Done.
Parsing results... Done.
Uploading data to Enrichr... Done.
  Querying MSigDB_Hallmark_2020... Done.
  Querying GO_Biological_Process_2021... Done.
Parsing results... Done.
Uploading data to Enrichr... Done.
  Querying MSigDB_Hallmark_2020... Done.
  Querying GO_Biological_Process_2021... Done.
Parsing results... Done.
Uploading data to Enrichr... Done.
  Querying MSigDB_Hallmark_2020... Done.
  Querying GO_Biological_Process_2021... Done.
Parsing results... Done.
Uploading data to Enrichr... Done.
  Querying MSigDB_Hallmark_2020... Done.
  Querying GO_Biological_Process_2021... Done.
Parsing results... Done.
Uploading data to Enrichr... Done.
  Querying MSigDB_Hallmark_2020... Done.
  Querying GO_Biological_Process_202

png 
  2

In [ ]:

h_mut_enr_167 <- h_mut_enr1


